### WE ARE READING NEWS BY USING GOOGLE NEWS API TO CALCULATE THE PRICES OF FACEBOOK MARKET BY ANALYZING THE TEXT DATA

In [1]:
#!pip install pyspellchecker
#!pip install regex
#!conda install -c anaconda nltk

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen
from urllib.request import Request
from GoogleNews import GoogleNews
#from pygooglenews import GoogleNews
from spellchecker import SpellChecker
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from spellchecker import SpellChecker
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np


C:\Users\Zigron\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Zigron\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Zigron\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


### Intialize all object that are to be the part of text cleaning and calculating sentiment score

In [3]:
spell = SpellChecker()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
sid_obj = SentimentIntensityAnalyzer()


## Google News Scraper

In [4]:
def google_news(searches, lang='en', period='1d'):
    final_df = pd.DataFrame(columns=['title', 'datetime'])
    for search in searches:
        googlenews = GoogleNews(lang=lang,period=period)
        googlenews.get_news(search)
        scraped_news = pd.DataFrame(googlenews.results())
        scraped_news = scraped_news.drop(['desc', 'link', 'date','img', 'media', 'site'], axis=1)
        final_df = final_df.append(scraped_news)
    return scraped_news, googlenews, final_df

In [5]:
_, news, scraped_news = google_news(['Facebook', 'Facebook Stock Price'])

In [6]:
scraped_news.head()

,title,datetime
0,Leaving Facebook? Easier said than done,2021-11-11 16:08:50.929485
1,The types of Facebook ads you see could change...,2021-11-11 02:08:50.929485
2,"On Hate In India, Facebook's Surprise Findings",2021-11-11 21:08:50.929485
3,Facebook Whistleblower Frances Haugen 'Extreme...,2021-11-11 15:08:50.929485
4,Facebook's work app Workplace to integrate wit...,2021-11-11 11:08:50.929485


## FinViz Scraper

In [7]:
def finviz_scraper(tickers, n):
    finviz_url = 'https://finviz.com/quote.ashx?t='
    news_tables = {}
    for ticker in tickers:
        url = finviz_url + ticker
        req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
        resp = urlopen(req)    
        html = BeautifulSoup(resp, features="lxml")
        news_table = html.find(id='news-table')
        news_tables[ticker] = news_table
        
    try:
        for ticker in tickers:
            df = news_tables[ticker]
            df_tr = df.findAll('tr')

            for i, table_row in enumerate(df_tr):
                a_text = table_row.a.text
                td_text = table_row.td.text
                td_text = td_text.strip()
                if i == n-1:
                    break
    except KeyError:
        pass
    
    parsed_news = []
    for file_name, news_table in news_tables.items():
        for x in news_table.findAll('tr'):
            text = x.a.get_text() 
            date_scrape = x.td.text.split()

            if len(date_scrape) == 1:
                time = date_scrape[0]

            else:
                date = date_scrape[0]
                time = date_scrape[1]

            ticker = file_name.split('_')[0]

            parsed_news.append([ticker, date, time, text])
    columns = ['Ticker', 'Date', 'Time', 'Headline']
    news = pd.DataFrame(parsed_news, columns=columns)
    news['DateTime'] = news['Date'] + ' ' + news['Time']
    news = news.drop(['Date', 'Time'], axis=1)
    news['DateTime'] = pd.to_datetime(news['DateTime'])
    return news,news_tables

In [8]:
fin_news, fin_table = finviz_scraper(['FB'], n=10)
fin_news['DateTime'] = fin_news['DateTime'].dt.date

### Merging Finviz and Google News Data

In [9]:
scraped_news['datetime'] = scraped_news['datetime'].dt.date
scraped_news.dropna(inplace=True)
scraped_news['Ticker'] = 'FB'

fin_news.columns = ['ticker', 'headline', 'datetime']
scraped_news.columns = ['headline', 'datetime', 'ticker']
fin_news = fin_news[['headline', 'datetime', 'ticker']]
full_df = fin_news.append(scraped_news)
full_df.sort_values(by='datetime', inplace=True)

### HERE WE CONCATENATE THE DATASET 

In [10]:
full_df.head()

,headline,datetime,ticker
99,FB Investor Alert: Shareholder Lawsuit Filed,2021-11-07,FB
80,"AMD partners with Meta, seeks to 'advance semi...",2021-11-08,FB
68,AMD Stock Is Surging. Facebooks Parent Is Usin...,2021-11-08,FB
69,"IMPORTANT REMINDER: FACEBOOK, INC. (NASDAQ:FB)...",2021-11-08,FB
70,Why AMD Stock Surged to a New All-Time High Today,2021-11-08,FB


###  We have to Check how many rows and columns in the dataset

In [11]:
rows,columns=full_df.shape
print('NUMBER OF ROWS :',rows,'','NUMBER OF COLUMNS :',columns)

NUMBER OF ROWS : 230  NUMBER OF COLUMNS : 3


###  This function is used to remove the redundant of the values present in the dataframe

In [12]:
def remove_redundant_rows(FacebookData) :
    return FacebookData.drop_duplicates()

### Cleaning the text by removing url hashtags mentions puntuation and extra whitespaces from text

In [13]:
def cleaning_text(sentence) :
    sentence = sentence.lower()                    # lower text
    sentence = re.sub('http\S+\s*', '', sentence)  # remove URLs
    sentence = re.sub('\W+', ' ', sentence)        # remove commas 
    sentence= re.sub('RT|cc', '', sentence)  # remove RT and cc
    sentence = re.sub('#\S+', '', sentence)  # remove hashtags
    sentence = re.sub('@\S+', '', sentence)  # remove mentions
    sentence = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), '',sentence)  # remove punctuations
    sentence = re.sub('\s+', ' ', sentence)  # remove extra whitespace
    sentence = re.sub(r'[0-9]', '', sentence) # remove digits from text
    return sentence
    
    
    

### Tokenization is used to remove the useless words thats are not much worthy for our sentiment analysis 

In [14]:
def tokenization_lemmatize_stopwording(sentence):
    sentence = word_tokenize(sentence)
    sentence = [lemmatizer.lemmatize(i) for i in sentence if not i in stop_words]
    sentence = ' '.join(sentence)
    return sentence

### I am using vader library to calculate sentiment score 

In [15]:
def sentiment_scores(sentence):
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)

    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        return "Positive"
    elif sentiment_dict['compound'] <= - 0.05 :
        return "Negative"
    else :
        return "Neutral"

### Here we can predict the price based on sentiment 

In [16]:
def price_prediction(sentiment) :
    if sentiment=='Positive' :
        return 'Increase'
    elif sentiment=='Negative' :
        return 'Negative'
    else :
        return 'Nochange'
    

### Convert list into DateFrame

In [17]:
def data_frame(list_) :
    return pd.DataFrame(list_, columns=['Date', 'Text','Textsentiment','Priceprediction'])
    

### DAY BY DAY  Text SENTIMENT  ANALYSIS

In [18]:
def sentiment_average_score(Dataframe_) :
    sc=pd.pivot_table(Dataframe_,index=['Date'],
                          columns=['Textsentiment'],aggfunc='count',fill_value=0)
    return sc

### DAY BY DAY  Price SENTIMENT  ANALYSIS

In [66]:
def Calculate_price_analysis_dy_by_day (pivot_table) :
    Dataframe=pivot_table['Priceprediction'].reset_index()
    for row in  Dataframe.itertuples(index = True, name ='Pandas'):
        print(getattr(row, "Date"))
        
    
    

### Text Preprocessing is the technique used clean the dataset

In [19]:
list_contain_sentiments=[]
def text_preprocessing (FacebookData) :
    
    FacebookData=remove_redundant_rows(FacebookData) # Redundunt data remove 
    for row in FacebookData.itertuples(index = True, name ='Pandas'):
        text= cleaning_text(getattr(row, "headline"))
        text=tokenization_lemmatize_stopwording(text)
        sentiment=sentiment_scores(text)
        pricesentiment=price_prediction(sentiment)
        
            
        list_contain_sentiments.append([getattr(row, "datetime"),text,sentiment,pricesentiment])
        
    return list_contain_sentiments
        

In [68]:
list_contain_sentiments=text_preprocessing(full_df)
dataframe=data_frame(list_contain_sentiments)
privot_table=sentiment_average_score(dataframe[['Date','Textsentiment','Priceprediction']])

In [69]:
Calculate_price_analysis_dy_by_day(privot_table)

2021-11-07
2021-11-08
2021-11-09
2021-11-10
2021-11-11
